In [1]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
import json


In [2]:
consumer_key = "xxxxxx"
consumer_secret = "xxxxxx"
acess_token = "xxxxxx"
acess_token_secret = "xxxxxx"

In [ ]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_acess_token(acess_token, acess_token_secret)
